In [ ]:
import os
ROOT = '/home/mav204/Documents/minor-project'
os.chdir(ROOT)
print(ROOT)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd drive/MyDrive/minor-project

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from misc.dataset_modifier import save_json, get_json
import torch
import json
import re
import json

/home/mav204/Documents/minor-project/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


misc.dataset_modifier 




In [ ]:
def build_prompt(sample):
    return f"""
You are a strict grader.

Return JSON only:
{{
  "predicted_label": "correct|partial|incorrect|uncertain",
  "confidence": 0.0-1.0
}}

Rules:
- Base your judgment primarily on the Reference Answer vs Provided Answer.
- Use the normalized_score only as supporting evidence.
- If content clearly contradicts the reference, ignore the score.
- correct: matches all key points
- partial: mostly correct, missing minor points
- incorrect: wrong or contradicts reference
- uncertain: ambiguous or insufficient info

Question: {sample['question']}
Reference Answer: {sample['reference_answer']}
Provided Answer: {sample['provided_answer']}
Original Label: {sample['verification_feedback']}
Normalized Score: {sample['normalized_score']}
"""

In [ ]:
model_id = "Qwen/Qwen2.5-14B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
def audit_sample(sample):
    prompt = build_prompt(sample)

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=120,
            do_sample=False,   # IMPORTANT
            temperature=0.0,
            top_p=1.0
        )

    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return text


In [ ]:
samples = get_json("./data/augmented/aug.json")

results = []
for s in samples:
    out = audit_sample(s)
    results.append({
        "id": s.get("id"),
        "raw_output": out
    })


NameError: name 'get_json' is not defined

In [ ]:

def extract_json(text):
    match = re.search(r"\{.*\}", text, re.S)
    return json.loads(match.group()) if match else None

cleaned = []
for r in results:
    parsed = extract_json(r["raw_output"])
    if parsed:
        cleaned.append(parsed)

In [ ]:
save_json(cleaned, './data/metadata/audit.json')